In [ ]:
# 1. 설치
!pip install fastapi uvicorn nest-asyncio pyngrok lightfm

# 2. 전체 코드 실행
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import pickle
import numpy as np
import nest_asyncio
from pyngrok import ngrok

# 3. 서버 충돌 방지
nest_asyncio.apply()
app = FastAPI()

# 4. 모델 로딩
with open("/content/lightfm_model.pt", "rb") as f:
    model = pickle.load(f)

with open("/content/user_map.pkl", "rb") as f:
    user_map = pickle.load(f)

with open("/content/item_ids.pkl", "rb") as f:
    item_ids = pickle.load(f)

# 5. 요청 형식
class RecommendRequest(BaseModel):
    user_id: int
    k: int = 6

# 6. 추천 API
@app.post("/recommend")
def recommend(req: RecommendRequest):
    user_id_raw = req.user_id
    if user_id_raw not in user_map:
        return {"recommended": []}

    user_idx = user_map[user_id_raw]
    scores = model.predict(user_ids=user_idx, item_ids=np.arange(len(item_ids)), num_threads=1)
    top_items_idx = np.argsort(-scores)[:req.k]
    top_product_ids = [int(item_ids[i]) for i in top_items_idx]
    return {"recommended": top_product_ids}

# 7. ngrok 인증 (최초 1회만)
!ngrok config add-authtoken 30ZZki80BWaaSqMmyq70kZKBxwc_5vJv2zxsRzdx44Kp8bbat

# 8. ngrok 연결
public_url = ngrok.connect(8000)
print(f"🔗 FastAPI 서버 주소: {public_url}/docs")

# 9. FastAPI 서버 실행 (실시간으로 Colab에서 실행)
uvicorn.run(app, host="0.0.0.0", port=8000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


INFO:     Started server process [2719]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🔗 FastAPI 서버 주소: NgrokTunnel: "https://48c85bcb408c.ngrok-free.app" -> "http://localhost:8000"/docs
INFO:     58.122.175.97:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     58.122.175.97:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     58.122.175.97:0 - "POST /recommend HTTP/1.1" 200 OK
